**Общая информация**

**Срок сдачи:** 20 марта 2019, 06:00 <br\>
**Штраф за опоздание:** -1 балл после 06:00 20 марта, -2 балла после 06:00 27 марта, -3 балла после 06:00 3 апреля

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack скинуть ссылку в личный чат @alkhamush
Пример ссылки:
https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 4 (3 балла)
Добавить algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Значение n_neighbors нужно взять из задания 3. Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

###### Задание 5 (1 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)

**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять структуру кода и текст. В противном случае -1 балл


In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist
import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации) Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn



In [20]:
# %%pycodestyle

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):

        self.n_neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):
        # nothing
        self.y_tr = y
        self.x_tr = X
        
        self.n_class = np.max(y)+1
        


    def predict(self, X):
        if self.algorithm == "kd_tree":
            tree = KDTree(self.x_tr)
            _, ind = tree.query(X, k=self.n_neighbors)
            classes = y_train[ind]
            
        elif self.algorithm == "brute":
            distanse = cdist(X, self.x_tr, metric='euclidean' )
            classes = y_train[np.argsort(distanse)[:,:self.n_neighbors]]
        
        Y_pred = np.zeros(len(X), dtype = self.y_tr.dtype)
        
        for i in range(len(X)):
            Y_pred[i] = np.argmax(np.bincount(classes[i]))

        return Y_pred

    def predict_proba(self, X):
        if self.algorithm == "kd_tree":
            tree = KDTree(self.x_tr)
            distanse, ind = tree.query(X, k=self.n_neighbors)
            classes = y_train[ind]
            
        elif self.algorithm == "brute":
            distanse = cdist(X, self.x_tr, metric='euclidean' )
            classes = y_train[np.argsort(distanse)[:,:self.n_neighbors]]
        
        Y_pred = np.zeros((len(X), self.n_class), dtype = np.float16)
        
        c = 0
        for i in classes:
            for j in i:
                Y_pred[c][j] += 1.0
            
            Y_pred[c] = Y_pred[c] / np.sum(Y_pred[c])
            
            c += 1
            

        return Y_pred
            
    def score(self, X, y):
        return (self.predict(X) == y).mean()

**IRIS**

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
from sklearn.neighbors import KDTree

In [121]:
tree = KDTree(X_train)

In [235]:
q = np.array([1, 0, 1])
q/sum(q)

array([0.5, 0. , 0.5])

In [226]:
clas = y_train[np.argsort(cdist(X_test, X_train, metric='euclidean' ))[:,:2]]
for i in clas:
    for j in i:
        print(j)

0
0
2
2
2
1
1
1
2
2
2
2
0
0
0
0
1
1
0
0
2
2
0
0
1
1
1
1
1
1


In [181]:
dist, ind = tree.query(X_test, k=3)

In [182]:
ind

array([[ 91,  59,  42],
       [ 51,   5,  74],
       [ 68, 112,  95],
       [ 49, 118,  18],
       [ 61,  98,  84],
       [ 38,  83,  28],
       [ 82,  14, 101],
       [ 12,   8,   1],
       [105,  86,  15],
       [ 83,  28,  23],
       [ 90,  32, 106],
       [ 59,  91,  67],
       [ 94,  23,  13],
       [111,  82,  75],
       [ 52,   0,  58]])

In [6]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [7]:
%time clf.fit(X_train, y_train)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 86.3 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [8]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 154 µs


In [9]:
%time clf.predict(X_test)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.05 ms


array([0, 2, 0, 2, 1, 1, 0, 0, 1, 0, 2, 2, 1, 1, 1])

In [10]:
%time my_clf.predict(X_test)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.38 ms


array([0, 2, 0, 2, 1, 1, 0, 0, 1, 0, 2, 2, 1, 1, 1])

In [11]:
%time clf.predict_proba(X_test)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.04 ms


array([[1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0.5, 0.5],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ]])

In [12]:
%time my_clf.predict_proba(X_test)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 3.17 ms


array([[1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0.5, 0.5],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ]], dtype=float16)

In [13]:
clf.score(X_test, y_test)

0.9333333333333333

In [14]:
my_clf.score(X_test, y_test)

0.9333333333333333

Задание 3

In [299]:
!gunzip t*-ubyte.gz

gzip: train-images-idx3-ubyte already exists; do you wish to overwrite (y or n)? ^C


**MNIST**

In [15]:
from mlxtend.data import loadlocal_mnist

In [16]:
X, y = loadlocal_mnist(
        images_path='train-images-idx3-ubyte', 
        labels_path='train-labels-idx1-ubyte')

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, stratify=y)

In [18]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [19]:
%time clf.fit(X_train, y_train)

CPU times: user 15.5 s, sys: 104 ms, total: 15.6 s
Wall time: 15.5 s


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [145]:
%time my_clf.fit(X_train, y_train)

CPU times: user 8 ms, sys: 44 ms, total: 52 ms
Wall time: 142 ms


In [146]:
%time clf.predict(X_test)

CPU times: user 47.3 s, sys: 24 ms, total: 47.4 s
Wall time: 47.4 s


array([0, 7, 0, 5, 7, 7, 0, 9, 4, 3, 0, 2, 3, 0, 4, 2, 7, 5, 0, 9, 2, 8,
       5, 3, 7, 0, 1, 6, 5, 5, 9, 7, 6, 4, 2, 0, 3, 4, 5, 8, 5, 3, 7, 4,
       9, 3, 1, 4, 4, 9, 9, 3, 6, 2, 0, 2, 6, 9, 4, 1, 6, 5, 9, 8, 9, 9,
       3, 0, 3, 1, 1, 0, 3, 6, 1, 0, 9, 6, 6, 1, 7, 6, 7, 7, 7, 2, 4, 7,
       6, 6, 8, 0, 9, 1, 4, 9, 2, 0, 5, 8, 9, 3, 7, 0, 6, 8, 2, 9, 2, 9,
       7, 2, 4, 9, 5, 7, 0, 1, 0, 3, 6, 8, 8, 1, 0, 4, 9, 0, 9, 5, 9, 2,
       9, 1, 3, 0, 0, 6, 4, 4, 2, 6, 1, 3, 9, 1, 1, 5, 4, 6, 0, 0, 0, 3,
       3, 9, 4, 8, 8, 0, 7, 2, 3, 7, 1, 4, 0, 0, 8, 8, 9, 4, 5, 6, 5, 7,
       0, 7, 8, 8, 1, 1, 4, 3, 4, 0, 5, 0, 7, 3, 6, 6, 4, 5, 8, 2, 7, 4,
       0, 1, 8, 0, 2, 2, 7, 0, 7, 2, 5, 4, 1, 8, 5, 5, 0, 0, 3, 4, 6, 3,
       4, 2, 7, 1, 1, 2, 1, 9, 1, 4, 4, 3, 2, 0, 9, 8, 1, 2, 7, 8, 7, 8,
       9, 2, 8, 9, 6, 9, 6, 5, 0, 5, 1, 9, 2, 0, 8, 6, 3, 4, 6, 8, 1, 6,
       8, 3, 7, 2, 5, 3, 7, 1, 4, 5, 9, 2, 5, 5, 8, 6, 0, 2, 4, 4, 7, 2,
       9, 7, 7, 1, 0, 4, 1, 5, 9, 1, 6, 6, 1, 9, 9,

In [147]:
%time my_clf.predict(X_test)

CPU times: user 29.8 s, sys: 916 ms, total: 30.8 s
Wall time: 41.7 s


array([0, 7, 0, 5, 7, 7, 0, 9, 4, 3, 0, 2, 3, 0, 4, 2, 7, 5, 0, 9, 2, 8,
       5, 3, 7, 0, 1, 6, 5, 5, 9, 7, 6, 4, 2, 0, 3, 4, 5, 8, 5, 3, 7, 4,
       9, 3, 1, 4, 4, 9, 9, 3, 6, 2, 0, 2, 6, 9, 4, 1, 6, 5, 9, 8, 9, 9,
       3, 0, 3, 1, 1, 0, 3, 6, 1, 0, 9, 6, 6, 1, 7, 6, 7, 7, 7, 2, 4, 7,
       6, 6, 8, 0, 9, 1, 4, 9, 2, 0, 5, 8, 9, 3, 7, 0, 6, 8, 2, 9, 2, 9,
       7, 2, 4, 9, 5, 7, 0, 1, 0, 3, 6, 8, 8, 1, 0, 4, 9, 0, 9, 5, 9, 2,
       9, 1, 3, 0, 0, 6, 4, 4, 2, 6, 1, 3, 9, 1, 1, 5, 4, 6, 0, 0, 0, 3,
       3, 9, 4, 8, 8, 0, 7, 2, 3, 7, 1, 4, 0, 0, 8, 8, 9, 4, 5, 6, 5, 7,
       0, 7, 8, 8, 1, 1, 4, 3, 4, 0, 5, 0, 7, 3, 6, 6, 4, 5, 8, 2, 7, 4,
       0, 1, 8, 0, 2, 2, 7, 0, 7, 2, 5, 4, 1, 8, 5, 5, 0, 0, 3, 4, 6, 3,
       4, 2, 7, 1, 1, 2, 1, 9, 1, 4, 4, 3, 2, 0, 9, 8, 1, 2, 7, 8, 7, 8,
       9, 2, 8, 9, 6, 9, 6, 5, 0, 5, 1, 9, 2, 0, 8, 6, 3, 4, 6, 8, 1, 6,
       8, 3, 7, 2, 5, 3, 7, 1, 4, 5, 9, 2, 5, 5, 8, 6, 0, 2, 4, 4, 7, 2,
       9, 7, 7, 1, 0, 4, 1, 5, 9, 1, 6, 6, 1, 9, 9,

In [148]:
%time clf.predict_proba(X_test)

CPU times: user 47.2 s, sys: 28 ms, total: 47.3 s
Wall time: 49.2 s


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [149]:
%time my_clf.predict_proba(X_test)

CPU times: user 29 s, sys: 356 ms, total: 29.3 s
Wall time: 29.6 s


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [150]:
clf.score(X_test, y_test)

0.955

In [151]:
my_clf.score(X_test, y_test)

0.955

Задание 4

In [287]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [288]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, stratify=y)

In [289]:
%time clf.fit(X_train, y_train)

CPU times: user 14.2 s, sys: 80 ms, total: 14.3 s
Wall time: 14.3 s


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [290]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 385 µs


In [291]:
%time clf.predict(X_test)

CPU times: user 47 s, sys: 4 ms, total: 47 s
Wall time: 47 s


array([1, 6, 5, 5, 8, 9, 4, 0, 2, 6, 4, 7, 9, 9, 3, 3, 4, 7, 6, 2, 2, 3,
       2, 6, 5, 1, 7, 0, 8, 7, 1, 0, 7, 7, 1, 9, 2, 2, 6, 0, 4, 1, 3, 8,
       4, 9, 8, 8, 0, 6, 0, 9, 5, 9, 5, 1, 2, 2, 0, 9, 6, 7, 4, 9, 4, 2,
       5, 5, 8, 5, 0, 4, 1, 4, 1, 9, 9, 8, 2, 2, 0, 0, 8, 9, 4, 9, 9, 5,
       7, 6, 9, 9, 0, 2, 7, 9, 0, 0, 2, 2, 4, 1, 8, 3, 3, 8, 2, 5, 8, 4,
       0, 5, 0, 1, 5, 9, 0, 6, 6, 6, 4, 4, 5, 8, 3, 3, 3, 4, 3, 6, 4, 3,
       7, 6, 9, 4, 0, 4, 0, 6, 3, 0, 3, 3, 7, 6, 8, 6, 5, 4, 8, 4, 1, 3,
       8, 5, 9, 7, 4, 3, 1, 1, 8, 2, 0, 7, 4, 7, 2, 9, 5, 4, 7, 8, 7, 7,
       5, 0, 8, 0, 6, 6, 9, 1, 1, 8, 7, 8, 3, 6, 9, 9, 4, 3, 6, 7, 2, 6,
       2, 1, 0, 3, 4, 8, 9, 3, 6, 0, 5, 4, 0, 3, 4, 0, 8, 4, 3, 0, 0, 3,
       1, 6, 1, 9, 4, 6, 7, 1, 8, 9, 1, 6, 1, 5, 2, 1, 1, 1, 9, 2, 4, 1,
       4, 4, 5, 3, 8, 6, 3, 7, 2, 1, 3, 0, 2, 2, 5, 5, 3, 2, 9, 6, 1, 8,
       6, 7, 6, 5, 1, 7, 7, 7, 4, 0, 8, 3, 1, 9, 7, 9, 2, 0, 2, 4, 7, 5,
       2, 3, 3, 4, 1, 3, 1, 7, 7, 7, 9, 7, 3, 9, 1,

In [292]:
%time my_clf.predict(X_test)

CPU times: user 1min 12s, sys: 308 ms, total: 1min 13s
Wall time: 1min 13s


array([1, 6, 5, 5, 8, 9, 4, 0, 2, 6, 4, 7, 9, 9, 3, 3, 4, 7, 6, 2, 2, 3,
       2, 6, 5, 1, 7, 0, 8, 7, 1, 0, 7, 7, 1, 9, 2, 2, 6, 0, 4, 1, 3, 8,
       4, 9, 8, 8, 0, 6, 0, 9, 5, 9, 5, 1, 2, 2, 0, 9, 6, 7, 4, 9, 4, 2,
       5, 5, 8, 5, 0, 4, 1, 4, 1, 9, 9, 8, 2, 2, 0, 0, 8, 9, 4, 9, 9, 5,
       7, 6, 9, 9, 0, 2, 7, 9, 0, 0, 2, 2, 4, 1, 8, 3, 3, 8, 2, 5, 8, 4,
       0, 5, 0, 1, 5, 9, 0, 6, 6, 6, 4, 4, 5, 8, 3, 3, 3, 4, 3, 6, 4, 3,
       7, 6, 9, 4, 0, 4, 0, 6, 3, 0, 3, 3, 7, 6, 8, 6, 5, 4, 8, 4, 1, 3,
       8, 5, 9, 7, 4, 3, 1, 1, 8, 2, 0, 7, 4, 7, 2, 9, 5, 4, 7, 8, 7, 7,
       5, 0, 8, 0, 6, 6, 9, 1, 1, 8, 7, 8, 3, 6, 9, 9, 4, 3, 6, 7, 2, 6,
       2, 1, 0, 3, 4, 8, 9, 3, 6, 0, 5, 4, 0, 3, 4, 0, 8, 4, 3, 0, 0, 3,
       1, 6, 1, 9, 4, 6, 7, 1, 8, 9, 1, 6, 1, 5, 2, 1, 1, 1, 9, 2, 4, 1,
       4, 4, 5, 3, 8, 6, 3, 7, 2, 1, 3, 0, 2, 2, 5, 5, 3, 2, 9, 6, 1, 8,
       6, 7, 6, 5, 1, 7, 7, 7, 4, 0, 8, 3, 1, 9, 7, 9, 2, 0, 2, 4, 7, 5,
       2, 3, 3, 4, 1, 3, 1, 7, 7, 7, 9, 7, 3, 9, 1,

In [293]:
%time clf.predict_proba(X_test)

CPU times: user 54.6 s, sys: 56 ms, total: 54.6 s
Wall time: 54.7 s


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [294]:
%time my_clf.predict_proba(X_test)

CPU times: user 1min 10s, sys: 192 ms, total: 1min 10s
Wall time: 1min 10s


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [295]:
clf.score(X_test, y_test)

0.9633333333333334

In [ ]:
my_clf.score(X_test, y_test)

Задание 5

In [ ]:
seaborn
matplotlib
train_test_split
Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
StandardScaler (from sklearn.preprocessing import StandardScaler)
ClassifierMixin
NeighborsBase
KNeighborsMixin
SupervisedIntegerMixin